#### Valores faltantes
Pandas maneja los valores faltantes usando el tipo None y los valores NaN de NumPy. Los valores faltantes son bastante comunes en las actividades de limpieza de datos.

Por ejemplo, si estás realizando una encuesta y un encuestado no respondió una pregunta, el valor faltante es en realidad una omisión. Este tipo de datos faltantes se llama Missing at Random (MAR) (Faltante al Azar) si hay otras variables que podrían usarse para predecir la variable que falta. En mi trabajo, cuando realizo encuestas, a menudo encuentro que los datos faltantes, como el interés en participar en un estudio de seguimiento, a menudo tienen alguna correlación con otro campo de datos, como el género o la etnia. Si no hay relación con otras variables, entonces llamamos a estos datos Missing Completely at Random (MCAR) (Faltante Completamente al Azar).

In [2]:
import pandas as pd

In [5]:
# pandas es muy bueno detectando datos faltantes. A pesar de que los datos faltantes a menudo
# se formatean como NAN, NULL, None or N/A, aveces los datos no estan marcados tan claramente. 

# la función de pandas read_csv() tiene un parámetro llamado na_values que nos permite especificar
# el formato de los valores faltantes
dataframe = pd.read_csv("../datasets/class_grades.csv")
dataframe.head()

,Prefix,Assignment,Tutorial,Midterm,TakeHome,Final
0,5,57.14,34.09,64.38,51.48,52.50
1,8,95.05,105.49,67.50,99.07,68.33
2,8,83.70,83.17,NaN,63.15,48.89
3,7,NaN,NaN,49.38,105.93,80.56
4,8,91.32,93.64,95.00,107.41,73.89


In [7]:
# podemos pasarle la función isnull() para crear una mascara booleana de todo el dataframe
# esto propaga la función isnull a todos los elementos del dataframe
mascara = dataframe.isnull()
mascara.head()

,Prefix,Assignment,Tutorial,Midterm,TakeHome,Final
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,True,False,False
3,False,True,True,False,False,False
4,False,False,False,False,False,False


In [8]:
# otra operación que podemos realizar es dropear (eliminar) las filas que contengan algun
# valor faltante, para esto utilizamos dropna()
dataframe.dropna().head()

,Prefix,Assignment,Tutorial,Midterm,TakeHome,Final
0,5,57.14,34.09,64.38,51.48,52.50
1,8,95.05,105.49,67.50,99.07,68.33
4,8,91.32,93.64,95.00,107.41,73.89
5,7,95.00,92.58,93.12,97.78,68.06
6,8,95.05,102.99,56.25,99.07,50.00


In [9]:
# podemos cambiar todos los valores faltantes por un valor concreto utilizando la función
# fillna(). Esta función toma un número de parametros, podemos pasarle un solo valor para 
# cambiar todos los datos faltantes a un valor.

# por ejemplo podemos querer rellenar todos los datos faltantes con un 0. Con inplace=True
# aplicamos los cambios sobre el dataframe en lugar de generar una copia
dataframe.fillna(0, inplace=True).head()

,Prefix,Assignment,Tutorial,Midterm,TakeHome,Final
0,5,57.14,34.09,64.38,51.48,52.50
1,8,95.05,105.49,67.50,99.07,68.33
2,8,83.70,83.17,0.00,63.15,48.89
3,7,0.00,0.00,49.38,105.93,80.56
4,8,91.32,93.64,95.00,107.41,73.89


In [11]:
# también podemos usar la opción na_filter, para desactivar el filtrado de espacios en blanco
# en grandes dataframes pasar na_filter=False, puede mejorar el rendimiento de lectura

# a veces es util considerar los valores faltantes como información efectiva, en el siguien 
# dataframe por ejemplo NaN se utiliza para representar que no hay cambios desde el registro 
# anterior
dataframe = pd.read_csv("../datasets/log.csv")
dataframe.head()

,time,user,video,playback position,paused,volume
0,1469974424,cheryl,intro.html,5,False,10.0
1,1469974454,cheryl,intro.html,6,NaN,NaN
2,1469974544,cheryl,intro.html,9,NaN,NaN
3,1469974574,cheryl,intro.html,10,NaN,NaN
4,1469977514,bob,intro.html,1,NaN,NaN


In [12]:
# en pandas podemos ordenar por index o por valor, vamos a promocionar los timestamps a 
# indices y entonces ordenaremos por indices
dataframe = dataframe.set_index("time")
dataframe = dataframe.sort_index()
dataframe.head()

,user,video,playback position,paused,volume
time,,,,,
1469974424,cheryl,intro.html,5,False,10.0
1469974424,sue,advanced.html,23,False,10.0
1469974454,cheryl,intro.html,6,NaN,NaN
1469974454,sue,advanced.html,24,NaN,NaN
1469974484,cheryl,intro.html,7,NaN,NaN


In [13]:
# podemos ver que el indice en este caso no es unico, es decir, dos usuarios pueden tener el
# mismo timestamp. 

# para obtener una visión más clara de los datos podemos usar indices multinivel con time y
# user, promocionando user a un indice de segundo nivel
dataframe = dataframe.reset_index()
dataframe = dataframe.set_index(["time", "user"])
dataframe.head()

video  playback position paused  volume
time       user                                                   
1469974424 cheryl     intro.html                  5  False    10.0
           sue     advanced.html                 23  False    10.0
1469974454 cheryl     intro.html                  6    NaN     NaN
           sue     advanced.html                 24    NaN     NaN
1469974484 cheryl     intro.html                  7    NaN     NaN

In [15]:
# ahora que tenemos los datos indexados y ordenados correctamente, podemos rellenar los datas
# faltantes utilizando ffill.
dataframe = dataframe.ffill()
dataframe.head()

video  playback position  paused  volume
time       user                                                    
1469974424 cheryl     intro.html                  5   False    10.0
           sue     advanced.html                 23   False    10.0
1469974454 cheryl     intro.html                  6   False    10.0
           sue     advanced.html                 24   False    10.0
1469974484 cheryl     intro.html                  7   False    10.0

In [17]:
# también podemos personalizar con que rellenamos los datos faltantes, utilizando la función
# replace(). Nos permite el reemplazo con varios enfoques: valor por valor, lists, diccionarios y
# regex...

dataframe = pd.DataFrame({'A': [1, 1, 2, 3, 4],
                   'B': [3, 6, 3, 8, 9],
                   'C': ['a', 'b', 'c', 'd', 'e']})
dataframe

,A,B,C
0,1,3,a
1,1,6,b
2,2,3,c
3,3,8,d
4,4,9,e


In [18]:
# reemplazar valor por valor
dataframe.replace(1, 100)

,A,B,C
0,100,3,a
1,100,6,b
2,2,3,c
3,3,8,d
4,4,9,e


In [20]:
# cambiar dos valores a la vez. 1 por 100 y 2 por 200
dataframe.replace([1, 2], [100, 200])

,A,B,C
0,100,3,a
1,100,6,b
2,200,3,c
3,3,8,d
4,4,9,e


In [23]:
# para reemplazar valores usando regex, pasamos el patron regex como primer parámetro
# el segundo parámetro será el valor por el que vamos a sustituir cuando coincidan y el 
# tercero "regex=True"
dataframe = pd.read_csv("../datasets/log.csv")

dataframe.replace(to_replace=".*.html$", value="página web", regex=True, inplace=True)
dataframe

,time,user,video,playback position,paused,volume
0,1469974424,cheryl,página web,5,False,10.0
1,1469974454,cheryl,página web,6,NaN,NaN
2,1469974544,cheryl,página web,9,NaN,NaN
3,1469974574,cheryl,página web,10,NaN,NaN
4,1469977514,bob,página web,1,NaN,NaN
5,1469977544,bob,página web,1,NaN,NaN
6,1469977574,bob,página web,1,NaN,NaN
7,1469977604,bob,página web,1,NaN,NaN
8,1469974604,cheryl,página web,11,NaN,NaN
9,1469974694,cheryl,página web,14,NaN,NaN
